# Programming Assignment 3 - Simple Linear versus Ridge Regression 

Your friend Bob just moved to Boston. He is a real estate agent who is trying to evaluate the prices of houses in the Boston area. He has been using a linear regression model but he wonders if he can improve his accuracy on predicting the prices for new houses. He comes to you for help as he knows that you're an expert in machine learning. 

As a pro, you suggest doing a *polynomial transformation*  to create a more flexible model, and performing ridge regression since having so many features compared to data points increases the variance. 

Bob, however, being a skeptic isn't convinced. He wants you to write a program that illustrates the difference in training and test costs for both linear and ridge regression on the same dataset. Being a good friend, you oblige and hence this assignment :) 

In this notebook, you are to explore the effects of ridge regression.  We will use a dataset that is part of the sklearn.dataset package.  Learn more at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [1]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold

###  Importing the dataset

In [2]:
# Import the boston dataset from sklearn
boston_data = load_boston()

In [3]:
#  Create X and Y variables - X holding the .data and Y holding .target 
X = boston_data.data
y = boston_data.target

#  Reshape Y to be a rank 2 matrix 
y = y.reshape(X.shape[0], 1)

# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])
# Printing out the features
print('The features: ', boston_data.feature_names)
# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])
#Observing the first 2 rows of the data
print(X[0:2])


#remove
print(y[0:2])


The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]
[[24. ]
 [21.6]]


We will also create polynomial feeatures for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [4]:
# Create a PolynomialFeatures object with degree = 2. 
# Transform X and save it into X_2. Simply copy Y into Y_2 
poly = PolynomialFeatures(degree=2)
X_2 = poly.fit_transform(X)
y_2 = y

In [5]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


# Your code goes here

In [6]:
# TODO - Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# TODO - Return w values

def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    
    w= np.dot(np.linalg.pinv(X_train),y_train)
    ##############
    return w

In [7]:
# TODO - Define the evaluate_err_ridge function.
# TODO - Return the train_error and test_error values


def evaluate_err(X_train, X_test, y_train, y_test, w): 
    
    #calculating train error -
    w_trans_x_train = np.dot(X_train,w)
    train_error = np.sum(np.power((w_trans_x_train - y_train),2)) / (X_train.shape[0])
    
    #calculating test error 
    w_trans_x_test = np.dot(X_test,w)
    test_error = np.sum(np.power((w_trans_x_test - y_test),2)) / (X_test.shape[0])
    
    
    ##############
    return train_error, test_error

In [8]:
# TODO - Finish writting the k_fold_cross_validation function. 
# TODO - Returns the average training error and average test error from the k-fold cross validation
# use Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        y_train_mean = np.mean(y_train)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean
        # scaling the data matrix
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # determine the training error and the test error
        #### TO-DO #####
        w=get_coeff_ridge_normaleq(X_train, y_train, alpha)
        train_error,test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
        total_E_val_test +=test_error
        total_E_val_train +=train_error   
    
    
       ##############
    E_val_test = total_E_val_test / k
    E_val_train = total_E_val_train/k
    return  E_val_test, E_val_train
    


In [9]:
def predict(X_train,y_train,alpha,test_data):
    
    #predict using polynomial regression 
    poly = PolynomialFeatures(degree=2)
    X_2 = poly.fit_transform(X)
    y_2 = y
    test_data = poly.fit_transform(test_data)
    
    #scaling data
    y_2_mean = np.mean(y_2)
    y_2_train = y_2 - y_2_mean
    # scaling the data matrix
    scaler = preprocessing.StandardScaler().fit(X_2)
    X_2_train = scaler.transform(X_2)
    X_test = scaler.transform(test_data)
    #getting coefficient 
    w=get_coeff_ridge_normaleq(X_2_train, y_2_train, alpha)
    print("parameters:",w)
    #calculating predicted value using polynomial
    predictedvalue = np.dot(X_test,w)
    
    
    return predictedvalue
    

In [10]:

# implement linear regression when alpha=0
alpha=0
avg_test_error,avg_train_error = k_fold_cross_validation(10, X, y,alpha)
print("\nLinear regression -")
print("Average Train error:",avg_train_error)
print("Average Test error:",avg_test_error)

#Finding the best alpha for ridge regression
alpha = np.logspace(1, 7, num=13)
alphatesterr=np.empty(shape=[0, 13])
for a in range(np.size(alpha)):
    test_err,train_err = k_fold_cross_validation(10, X, y,alpha[a])
    alphatesterr = np.append(alphatesterr,test_err)

#finding the alpha value for minimum test error rate     
minimum_test_error_value=np.argmin(alphatesterr)
ind=np.unravel_index(minimum_test_error_value,alphatesterr.shape)
bestalpha=alpha[ind]
print("best alpha with minimum test error rate:",bestalpha)

#implementing ridge regression with best alpha
print("\nRidge regression -")
avg_test_error_ridge,avg_train_error_ridge = k_fold_cross_validation(10, X, y,bestalpha)
print("Average Train error for ridge regression:",avg_train_error_ridge)
print("Average Test error for ridge regression:",avg_test_error_ridge)

#implementing polynomial regression of degree 2 
avg_test_error_p,avg_train_error_p = k_fold_cross_validation(10, X_2, y_2,0)
print("\nPolynomial regression fo degree 2 -")
print("Average Train error for Polynomial regression:",avg_train_error_p)
print("Average Test error for polynomial regression:",avg_test_error_p)

#Finding the best alpha for polynomial regression
alpha1 = np.logspace(1, 7, num=105)
#print("alpha:",alpha)
alphatesterr1=np.empty(shape=[0, 105])
for a in range(np.size(alpha1)):
    test_err,train_err = k_fold_cross_validation(10, X, y,alpha1[a])
    alphatesterr1 = np.append(alphatesterr1,test_err)

#finding the alpha value for minimum test error rate     
minimum_test_error_value_p=np.argmin(alphatesterr1)
ind_p=np.unravel_index(minimum_test_error_value_p,alphatesterr1.shape)
bestalpha_p=alpha1[ind_p]
print("best alpha with minimum test error rate for polynomial regression:",bestalpha_p)
#implementing polynomial regression with regularization 
avg_test_error_pr,avg_train_error_pr = k_fold_cross_validation(10, X_2, y_2,bestalpha_p)
print("\nPolynomial regression for degree 2 with regularization-")
print("Average Train error for Polynomial regression:",avg_train_error_pr)
print("Average Test error for polynomial regression:",avg_test_error_pr)



#predicting prices -
#using polynomial regression for predicting price for given test data
test_data=np.array([[5,0.5, 2, 0, 4, 8, 4, 6, 2, 2, 2, 4, 5.5]])
print("\nFor predicting using Polynomial regression, alpha used is-",bestalpha_p)
print("\nPredicted value for test data is:",predict(X,y,bestalpha_p,test_data)[0])





Linear regression -
Average Train error: 21.806183575851065
Average Test error: 23.63606860542818
best alpha with minimum test error rate: 10.0

Ridge regression -
Average Train error for ridge regression: 21.806183575851065
Average Test error for ridge regression: 23.63606860542818

Polynomial regression fo degree 2 -
Average Train error for Polynomial regression: 5.808820816012467
Average Test error for polynomial regression: 11.854968235566679
best alpha with minimum test error rate for polynomial regression: 10.0

Polynomial regression for degree 2 with regularization-
Average Train error for Polynomial regression: 5.808820816012467
Average Test error for polynomial regression: 11.854968235566679

For predicting using Polynomial regression, alpha used is- 10.0
parameters: [[ 5.45404395e-12]
 [-3.91175879e+01]
 [ 4.81627018e+00]
 [-3.18362779e+01]
 [ 7.53954659e+00]
 [ 1.54958309e+01]
 [ 1.52221935e+01]
 [ 2.55289252e+01]
 [-1.57410181e+01]
 [ 1.69692008e+01]
 [ 3.65030542e+00]
 [ 